In [3]:
import pandas as pd
import MeCab
import json
import copy
from tqdm import tqdm_notebook as tqdm

In [4]:
mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

----------------------------------------------------------



Failed initializing MeCab. Please see the README for possible solutions:

    https://github.com/SamuraiT/mecab-python3#common-issues

If you are still having trouble, please file an issue here, and include the
ERROR DETAILS below:

    https://github.com/SamuraiT/mecab-python3/issues

issueを英語で書く必要はありません。

------------------- ERROR DETAILS ------------------------
arguments: -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd
error message: [ifs] no such file or directory: /usr/local/lib/mecab/dic/mecab-ipadic-neologd\dicrc


RuntimeError: 

In [4]:
data = pd.read_excel('dataset.xlsx', engine='openpyxl')

In [5]:
data

,ID,サンプルID,サンプル 問い合わせ文,サンプル 応答文,カテゴリ1,カテゴリ2,出典,<参考>UMカテゴリタグ,<参考>ＵＭサービスメニュー\n（標準的な行政サービス名称）
0,1,1,母子手帳を受け取りたいのですが、手続きを教えてください。,窓口で妊娠届をご記入いただき、母子手帳をお渡しします。\n住民票の世帯が別の方が代理で窓口に...,妊娠・出産,妊娠,NaN,妊娠・出産,妊娠の届出・母子健康手帳の交付
1,2,2,母子手帳の受け取り場所はどこですか？,母子手帳は、○○市役所本庁舎△△階××課窓口、◎◎出張所、………（その他の受け取り場所を適宜...,妊娠・出産,妊娠,NaN,妊娠・出産,妊娠の届出・母子健康手帳の交付
2,3,3,母子手帳はすぐに発行してもらえますか？,母子手帳は、妊娠届の内容を確認させていただき、その場でお渡しします。\n\n▼詳しくはこちら...,妊娠・出産,妊娠,NaN,妊娠・出産,妊娠の届出・母子健康手帳の交付
3,4,4,妊婦健診受診票はいつの分から使えますか？,妊婦健診の受診票は、受診票を受け取った日より後で、病院が妊婦健診と規定した日に利用できます。...,妊娠・出産,健診,NaN,妊娠・出産,妊婦健康診査
4,5,5,妊婦健診受診票は○○市外で使えますか？,妊婦健診の受診票は、●●県内の契約医療機関でお使いいただけます。\n受診希望の病院にお問い合...,妊娠・出産,健診,NaN,妊娠・出産,妊婦健康診査
...,...,...,...,...,...,...,...,...,...
657,975,658,帰宅困難者の避難施設はどこですか？,帰宅困難者のための支援（受入）施設についてはこちらをご覧ください。\n（自治体HP内関連ペー...,防災・防犯,NaN,NaN,防災・災害,帰宅困難者対策
658,976,659,住民税の納税証明書を取りたい。,住民税の納税証明書は、（申請場所（自治体の担当課、支所・出張所等））で発行しています。\n申...,税金,住民税,NaN,税,市税の証明・閲覧
659,977,660,軽自動車税の納税証明書を取りたい。,軽自動車税の納税証明書は、（申請場所（自治体の担当課、支所・出張所等））で発行しています。\...,税金,軽自動車税,NaN,税,軽自動車税
660,979,661,禁煙・分園の公園はどこですか？,○○市では園内の受動喫煙防止対策として分煙環境を整備するため、市立公園○か所を、禁煙公園また...,施設,区立公園,NaN,NaN,NaN


In [6]:
questions = list(data['サンプル 問い合わせ文'])
questions[:5]

['母子手帳を受け取りたいのですが、手続きを教えてください。',
 '母子手帳の受け取り場所はどこですか？',
 '母子手帳はすぐに発行してもらえますか？',
 '妊婦健診受診票はいつの分から使えますか？',
 '妊婦健診受診票は○○市外で使えますか？']

In [7]:
answers = list(data['サンプル 応答文'])
answers = [a.replace('\n\n', '\n') for a in answers]
answers[:5]

['窓口で妊娠届をご記入いただき、母子手帳をお渡しします。\n住民票の世帯が別の方が代理で窓口に来られる場合は、委任状が必要になります。\n▼詳しくはこちら\n（自治体HP内関連ページのURL）',
 '母子手帳は、○○市役所本庁舎△△階××課窓口、◎◎出張所、………（その他の受け取り場所を適宜記載）………で受け取れます。\n▼詳しくはこちら\n（自治体HP内関連ページのURL）',
 '母子手帳は、妊娠届の内容を確認させていただき、その場でお渡しします。\n▼詳しくはこちら\n（自治体HP内関連ページのURL）',
 '妊婦健診の受診票は、受診票を受け取った日より後で、病院が妊婦健診と規定した日に利用できます。\n▼詳しくはこちら\n（自治体HP内関連ページのURL）',
 '妊婦健診の受診票は、●●県内の契約医療機関でお使いいただけます。\n受診希望の病院にお問い合わせください。\n▼詳しくはこちら\n（自治体HP内関連ページのURL）']

In [8]:
print(mecab.parse(questions[0]))

母子手帳	名詞,固有名詞,一般,*,*,*,母子手帳,ボシテチョウ,ボシテチョー
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
受け取り	動詞,自立,*,*,五段・ラ行,連用形,受け取る,ウケトリ,ウケトリ
たい	助動詞,*,*,*,特殊・タイ,基本形,たい,タイ,タイ
の	名詞,非自立,一般,*,*,*,の,ノ,ノ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
が	助詞,接続助詞,*,*,*,*,が,ガ,ガ
、	記号,読点,*,*,*,*,、,、,、
手続き	名詞,サ変接続,*,*,*,*,手続き,テツヅキ,テツズキ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
教え	動詞,自立,*,*,一段,連用形,教える,オシエ,オシエ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
ください	動詞,非自立,*,*,五段・ラ行特殊,命令ｉ,くださる,クダサイ,クダサイ
。	記号,句点,*,*,*,*,。,。,。
EOS



In [76]:
tags = []
tag2cnt = {}
idx = 0

for question in questions:
    lines = mecab.parse(question).split('\n')[:-2]
    words = list(set([line.split('\t')[0] for line in lines if line.split('\t')[1].split(',')[0]=='名詞' and len(line.split('\t')[0])>1]))
    for w in words:
        if w not in list(tag2cnt.keys()):
            tag2cnt[w] = 1
        else:
            tag2cnt[w] += 1
    tag = [idx]
    tag.extend(words)
    tags.append(tag)
    idx += 1

In [77]:
tags

[[0, '手続き', '母子手帳'],
 [1, '受け取り', '場所', 'どこ', '母子手帳'],
 [2, '発行', '母子手帳'],
 [3, 'いつの', '妊婦健診', '受診'],
 [4, '市外', '妊婦健診', '受診'],
 [5, '受取', 'た分', '還付', '受診', '病院', '妊婦健診'],
 [6, '以上', '追加', '助成', '14回', '妊婦健診'],
 [7, '妊婦健診', '県内', '受診'],
 [8, '受診', '県内', '子ども'],
 [9, '受診', '妊婦健診', '県外'],
 [10, '子ども', '受診', '県外'],
 [11, '換金', '余り', '妊婦健診', '受診'],
 [12, '里帰り出産', '受診'],
 [13, '手続き', '必要', '子ども'],
 [14, '使い方', '助成', '乳房', 'ケア'],
 [15, '赤ちゃん', '訪問', '日時', '変更', '案内'],
 [16, '助成', '1か月'],
 [17, '指定', '案内', '子ども'],
 [18, '時間', '子ども'],
 [19, '6か月', '指定', '保健', '保健所', '以外', '乳児', '相談', '受診', '3歳', '歯科', '1歳'],
 [20, '7か月'],
 [21, '10か月'],
 [22, '期間内', '7か月', '受診'],
 [23, '10か月', '期間内', '受診'],
 [24, '市外', '7か月', '受診'],
 [25, '市外', '10か月', '受診'],
 [26, '市外', '6か月', '1歳', '受診'],
 [27, '保健所', '1歳'],
 [28, '3歳'],
 [29, '発行', '予防接種', '記録'],
 [30, '予防接種', '転入', '予診', '記録'],
 [31, '予防接種', '里帰り'],
 [32, '予防接種', '時期', '法定', '接種', '期間'],
 [33, '予防接種', '記録', 'インフルエンザ', 'おたふくかぜ'],
 [34, '産前', 'こと', '産後'],
 [

In [78]:
tag2cnt

{'手続き': 33,
 '母子手帳': 7,
 '受け取り': 2,
 '場所': 23,
 'どこ': 31,
 '発行': 9,
 'いつの': 1,
 '妊婦健診': 12,
 '受診': 20,
 '市外': 13,
 '受取': 1,
 'た分': 1,
 '還付': 1,
 '病院': 9,
 '以上': 1,
 '追加': 1,
 '助成': 20,
 '14回': 1,
 '県内': 2,
 '子ども': 70,
 '県外': 3,
 '換金': 1,
 '余り': 1,
 '里帰り出産': 2,
 '必要': 31,
 '使い方': 1,
 '乳房': 3,
 'ケア': 3,
 '赤ちゃん': 6,
 '訪問': 3,
 '日時': 1,
 '変更': 7,
 '案内': 4,
 '1か月': 1,
 '指定': 2,
 '時間': 8,
 '6か月': 5,
 '保健': 3,
 '保健所': 3,
 '以外': 2,
 '乳児': 5,
 '相談': 22,
 '3歳': 5,
 '歯科': 4,
 '1歳': 7,
 '7か月': 3,
 '10か月': 3,
 '期間内': 2,
 '予防接種': 28,
 '記録': 6,
 '転入': 7,
 '予診': 1,
 '里帰り': 1,
 '時期': 6,
 '法定': 1,
 '接種': 2,
 '期間': 3,
 'インフルエンザ': 4,
 'おたふくかぜ': 1,
 '産前': 2,
 'こと': 11,
 '産後': 1,
 '市役所': 12,
 '医師': 1,
 '申請': 58,
 '診断書': 1,
 '上乗せ': 1,
 '2000円': 1,
 '毎回': 1,
 '2歳': 1,
 '療育': 8,
 '医療': 40,
 '手続': 10,
 'もの': 11,
 'いつ': 17,
 '直接': 3,
 '育成': 8,
 '養育': 7,
 '特定疾病': 9,
 '慢性': 9,
 '小児': 10,
 '児童手当': 40,
 '申請方法': 3,
 '出生': 2,
 '第2子': 2,
 '転出': 6,
 '市内': 16,
 '引っ越し': 4,
 '証明': 5,
 '加入': 1,
 '厚生年金': 1,
 '別居': 2,
 '夫婦': 

In [79]:
# version 1
with open('document_v1.txt', 'w') as f:
    for i in range(len(questions)):
        f.write('<' + '・'.join(tags[i][1:]) + '>\n' + answers[i] + '\n\n')

In [80]:
# タグ並び替え
for tag in tags:
    new_tag = [tag[0]]
    for i in range(1, len(tag)-1):
        for j in range(i, len(tag)):
            if tag2cnt[tag[j]] > tag2cnt[tag[i]]:
                tag[i], tag[j] = tag[j], tag[i]
            elif tag2cnt[tag[j]] == tag2cnt[tag[i]] and tag[j] < tag[i]:
                tag[i], tag[j] = tag[j], tag[i]

In [81]:
tags

[[0, '手続き', '母子手帳'],
 [1, 'どこ', '場所', '母子手帳', '受け取り'],
 [2, '発行', '母子手帳'],
 [3, '受診', '妊婦健診', 'いつの'],
 [4, '受診', '市外', '妊婦健診'],
 [5, '受診', '妊婦健診', '病院', 'た分', '受取', '還付'],
 [6, '助成', '妊婦健診', '14回', '以上', '追加'],
 [7, '受診', '妊婦健診', '県内'],
 [8, '子ども', '受診', '県内'],
 [9, '受診', '妊婦健診', '県外'],
 [10, '子ども', '受診', '県外'],
 [11, '受診', '妊婦健診', '余り', '換金'],
 [12, '受診', '里帰り出産'],
 [13, '子ども', '手続き', '必要'],
 [14, '助成', 'ケア', '乳房', '使い方'],
 [15, '変更', '赤ちゃん', '案内', '訪問', '日時'],
 [16, '助成', '1か月'],
 [17, '子ども', '案内', '指定'],
 [18, '子ども', '時間'],
 [19, '相談', '受診', '1歳', '3歳', '6か月', '乳児', '歯科', '保健', '保健所', '以外', '指定'],
 [20, '7か月'],
 [21, '10か月'],
 [22, '受診', '7か月', '期間内'],
 [23, '受診', '10か月', '期間内'],
 [24, '受診', '市外', '7か月'],
 [25, '受診', '市外', '10か月'],
 [26, '受診', '市外', '1歳', '6か月'],
 [27, '1歳', '保健所'],
 [28, '3歳'],
 [29, '予防接種', '発行', '記録'],
 [30, '予防接種', '転入', '記録', '予診'],
 [31, '予防接種', '里帰り'],
 [32, '予防接種', '時期', '期間', '接種', '法定'],
 [33, '予防接種', '記録', 'インフルエンザ', 'おたふくかぜ'],
 [34, 'こと', '産前', '産後'],
 [

In [82]:
# タグのソート
max_tag_length = -1
for tag in tags:
    if len(tag) > max_tag_length:
        max_tag_length = len(tag)
for tag in tags:
    while len(tag) < max_tag_length:
        tag.append('')

In [83]:
max_tag_length

15

In [84]:
tags

[[0, '手続き', '母子手帳', '', '', '', '', '', '', '', '', '', '', '', ''],
 [1, 'どこ', '場所', '母子手帳', '受け取り', '', '', '', '', '', '', '', '', '', ''],
 [2, '発行', '母子手帳', '', '', '', '', '', '', '', '', '', '', '', ''],
 [3, '受診', '妊婦健診', 'いつの', '', '', '', '', '', '', '', '', '', '', ''],
 [4, '受診', '市外', '妊婦健診', '', '', '', '', '', '', '', '', '', '', ''],
 [5, '受診', '妊婦健診', '病院', 'た分', '受取', '還付', '', '', '', '', '', '', '', ''],
 [6, '助成', '妊婦健診', '14回', '以上', '追加', '', '', '', '', '', '', '', '', ''],
 [7, '受診', '妊婦健診', '県内', '', '', '', '', '', '', '', '', '', '', ''],
 [8, '子ども', '受診', '県内', '', '', '', '', '', '', '', '', '', '', ''],
 [9, '受診', '妊婦健診', '県外', '', '', '', '', '', '', '', '', '', '', ''],
 [10, '子ども', '受診', '県外', '', '', '', '', '', '', '', '', '', '', ''],
 [11, '受診', '妊婦健診', '余り', '換金', '', '', '', '', '', '', '', '', '', ''],
 [12, '受診', '里帰り出産', '', '', '', '', '', '', '', '', '', '', '', ''],
 [13, '子ども', '手続き', '必要', '', '', '', '', '', '', '', '', '', '', ''],
 [14

In [85]:
tags_sorted = sorted(tags, key=lambda x:(x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13], x[14]))

In [86]:
tags_sorted

[[503, '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 [588, '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 [21, '10か月', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 [631, '1歳', '6か月', '', '', '', '', '', '', '', '', '', '', '', ''],
 [27, '1歳', '保健所', '', '', '', '', '', '', '', '', '', '', '', ''],
 [28, '3歳', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 [162, '3歳', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 [495, '3歳', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 [497, '5才', '息子', '最近', '', '', '', '', '', '', '', '', '', '', ''],
 [20, '7か月', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 [585, 'ADHD', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 [419, 'BCG', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 [530, 'LGBT', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 [591, 'QRコード', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 [437, 'いつ', '', '', '', '', '', '', '', '', '', '', '', '', ''

In [87]:
def indent(n):
    indent_str = ''
    for i in range(n):
        indent_str += '\t'
    return indent_str

In [88]:
cnt_leaves = 0

In [89]:
# version 2
with open('document_v2.txt', 'w') as f:
    tmp_tag = []
    for tag in tags_sorted:
        tag_valid = [word for word in tag if word != '']
        return_cnt = 0
        for i in range(len(tmp_tag)):
            #print(tmp_tag, tag_valid)
            if i >= len(tag_valid)-1:
                break
            if tmp_tag[i] != tag_valid[i+1]:
                return_cnt += len(tmp_tag)-i
                break
        if return_cnt > 0:
            cnt_leaves += 1
            tmp_tag = tmp_tag[:-return_cnt]
        if len(tmp_tag)==0:
            f.write('\n')
        if len(tag_valid)>1:
            for word in tag_valid[1:]:
                if word not in tmp_tag:
                    tmp_tag.append(word)
                    f.write(indent(len(tmp_tag)-1) + '<' + word + '>\n')
        print(tmp_tag)
        f.write(indent(len(tmp_tag)-1) + answers[tag_valid[0]].replace('\n', '\n'+indent(len(tmp_tag)-1)) + '\n')

[]
[]
['10か月']
['1歳', '6か月']
['1歳', '保健所']
['3歳']
['3歳']
['3歳']
['5才', '息子', '最近']
['7か月']
['ADHD']
['BCG']
['LGBT']
['QRコード']
['いつ']
['いつ', '届出', '出生届']
['いつ', '市役所']
['いつ', '市役所', '年内']
['おしめ']
['がん検診']
['こと']
['こと', 'うち']
['こと', '産前', '産後']
['こども食堂']
['ごみ']
['ごみ']
['ごみ']
['ごみ', '分別']
['ご当地グルメ']
['とき', '児童扶養手当証書']
['とき', '入院', '食事', '療養']
['とき', '問い合わせ', '病気']
['ところ', 'リトミック']
['ところ', '放課後', '学童']
['どこ', '一時集合場所']
['どこ', '乳児']
['どこ', '予防接種']
['どこ', '公園', '分園', '禁煙']
['どこ', '出張所', '支所']
['どこ', '場所', '母子手帳', '受け取り']
['どこ', '妊婦健診', '乳児', '医療機関', '契約']
['どこ', '市内', '認定こども園']
['どこ', '戸籍']
['どこ', '支援', '子育て', 'センター', '家庭']
['どこ', '施設', 'ところ', '赤ちゃん', '体重', 'スケール', 'ベビー']
['どこ', '施設', '市民', '保養']
['どこ', '施設', '帰宅困難者', '避難']
['どこ', '病院', '乳腺炎']
['どこ', '相談', 'いじめ']
['どこ', '相談', 'しし']
['どこ', '相談', '問題', '近隣']
['どこ', '相談', '困窮', '生活']
['どこ', '相談', '子育て']
['どこ', '相談', '学習障害']
['どこ', '相談', '野良犬']
['どこ', '納税証明書', '固定資産税']
['どこ', '自治体', '名称', '実施', '子ども食堂']
['どこ', '避難場所']
['どこ', '避難所']
['なん', 'マイナポ

In [90]:
cnt_leaves

598

In [91]:
root_words = []

for tag in tags:
    if len(tag)<2:
        continue
    if tag[1] not in root_words:
        root_words.append(tag[1])

In [92]:
len(root_words), root_words[:10]

(172, ['手続き', 'どこ', '発行', '受診', '助成', '子ども', '変更', '相談', '7か月', '10か月'])

In [93]:
sample_cnt = {}
idx2tags = {}
for tag in tags_sorted:
    words = [word for word in tag if word != ''][1:]
    tmp = []
    key = ''
    for w in words:
        tmp.append(w)
        key = '>'.join(tmp)
        if key not in list(sample_cnt.keys()):
            sample_cnt[key] = 1
        else:
            sample_cnt[key] += 1
    if len(words)==0:
        key = ''
        if key not in list(sample_cnt.keys()):
            sample_cnt[key] = 1
        else:
            sample_cnt[key] += 1
    idx2tags[int(tag[0])] = key

In [94]:
sample_cnt

{'': 2,
 '10か月': 1,
 '1歳': 2,
 '1歳>6か月': 1,
 '1歳>保健所': 1,
 '3歳': 3,
 '5才': 1,
 '5才>息子': 1,
 '5才>息子>最近': 1,
 '7か月': 1,
 'ADHD': 1,
 'BCG': 1,
 'LGBT': 1,
 'QRコード': 1,
 'いつ': 4,
 'いつ>届出': 1,
 'いつ>届出>出生届': 1,
 'いつ>市役所': 2,
 'いつ>市役所>年内': 1,
 'おしめ': 1,
 'がん検診': 1,
 'こと': 3,
 'こと>うち': 1,
 'こと>産前': 1,
 'こと>産前>産後': 1,
 'こども食堂': 1,
 'ごみ': 4,
 'ごみ>分別': 1,
 'ご当地グルメ': 1,
 'とき': 3,
 'とき>児童扶養手当証書': 1,
 'とき>入院': 1,
 'とき>入院>食事': 1,
 'とき>入院>食事>療養': 1,
 'とき>問い合わせ': 1,
 'とき>問い合わせ>病気': 1,
 'ところ': 2,
 'ところ>リトミック': 1,
 'ところ>放課後': 1,
 'ところ>放課後>学童': 1,
 'どこ': 25,
 'どこ>一時集合場所': 1,
 'どこ>乳児': 1,
 'どこ>予防接種': 1,
 'どこ>公園': 1,
 'どこ>公園>分園': 1,
 'どこ>公園>分園>禁煙': 1,
 'どこ>出張所': 1,
 'どこ>出張所>支所': 1,
 'どこ>場所': 1,
 'どこ>場所>母子手帳': 1,
 'どこ>場所>母子手帳>受け取り': 1,
 'どこ>妊婦健診': 1,
 'どこ>妊婦健診>乳児': 1,
 'どこ>妊婦健診>乳児>医療機関': 1,
 'どこ>妊婦健診>乳児>医療機関>契約': 1,
 'どこ>市内': 1,
 'どこ>市内>認定こども園': 1,
 'どこ>戸籍': 1,
 'どこ>支援': 1,
 'どこ>支援>子育て': 1,
 'どこ>支援>子育て>センター': 1,
 'どこ>支援>子育て>センター>家庭': 1,
 'どこ>施設': 3,
 'どこ>施設>ところ': 1,
 'どこ>施設>ところ>赤ちゃん': 1,
 'どこ>施設>ところ>赤ちゃん>体重

In [124]:
idx2tags

{503: '',
 588: '',
 21: '10か月',
 631: '1歳>6か月',
 27: '1歳>保健所',
 28: '3歳',
 162: '3歳',
 495: '3歳',
 497: '5才>息子>最近',
 20: '7か月',
 585: 'ADHD',
 419: 'BCG',
 530: 'LGBT',
 591: 'QRコード',
 437: 'いつ',
 427: 'いつ>届出>出生届',
 118: 'いつ>市役所',
 587: 'いつ>市役所>年内',
 500: 'おしめ',
 236: 'がん検診',
 573: 'こと',
 499: 'こと>うち',
 34: 'こと>産前>産後',
 405: 'こども食堂',
 154: 'ごみ',
 392: 'ごみ',
 439: 'ごみ',
 527: 'ごみ>分別',
 570: 'ご当地グルメ',
 216: 'とき>児童扶養手当証書',
 195: 'とき>入院>食事>療養',
 524: 'とき>問い合わせ>病気',
 244: 'ところ>リトミック',
 466: 'ところ>放課後>学童',
 654: 'どこ>一時集合場所',
 620: 'どこ>乳児',
 525: 'どこ>予防接種',
 660: 'どこ>公園>分園>禁煙',
 401: 'どこ>出張所>支所',
 1: 'どこ>場所>母子手帳>受け取り',
 178: 'どこ>妊婦健診>乳児>医療機関>契約',
 121: 'どこ>市内>認定こども園',
 448: 'どこ>戸籍',
 400: 'どこ>支援>子育て>センター>家庭',
 508: 'どこ>施設>ところ>赤ちゃん>体重>スケール>ベビー',
 447: 'どこ>施設>市民>保養',
 657: 'どこ>施設>帰宅困難者>避難',
 598: 'どこ>病院>乳腺炎',
 156: 'どこ>相談>いじめ',
 593: 'どこ>相談>しし',
 641: 'どこ>相談>問題>近隣',
 626: 'どこ>相談>困窮>生活',
 625: 'どこ>相談>子育て',
 629: 'どこ>相談>学習障害',
 592: 'どこ>相談>野良犬',
 616: 'どこ>納税証明書>固定資産税',
 455: 'どこ>自治体>名称>実施>子ども食堂',

In [96]:
with open('tag2count.json', 'w') as f:
    json.dump(sample_cnt, f, indent=4, ensure_ascii=False)

In [97]:
len(list(sample_cnt.keys()))

1423

In [98]:
with open('idx2tag.json', 'w') as f:
    json.dump(idx2tags, f, indent=4, ensure_ascii=False)

In [153]:
# js = open('tag2count.json', 'r').read()
# json.loads(js)

{'10か月': 1,
 '1歳': 2,
 '1歳>6か月': 1,
 '1歳>保健所': 1,
 '3歳': 3,
 '5才': 1,
 '5才>息子': 1,
 '5才>息子>最近': 1,
 '7か月': 1,
 'ADHD': 1,
 'BCG': 1,
 'LGBT': 1,
 'QRコード': 1,
 'いつ': 4,
 'いつ>届出': 1,
 'いつ>届出>出生届': 1,
 'いつ>市役所': 2,
 'いつ>市役所>年内': 1,
 'おしめ': 1,
 'がん検診': 1,
 'こと': 3,
 'こと>うち': 1,
 'こと>産前': 1,
 'こと>産前>産後': 1,
 'こども食堂': 1,
 'ごみ': 4,
 'ごみ>分別': 1,
 'ご当地グルメ': 1,
 'とき': 3,
 'とき>児童扶養手当証書': 1,
 'とき>入院': 1,
 'とき>入院>食事': 1,
 'とき>入院>食事>療養': 1,
 'とき>問い合わせ': 1,
 'とき>問い合わせ>病気': 1,
 'ところ': 2,
 'ところ>リトミック': 1,
 'ところ>放課後': 1,
 'ところ>放課後>学童': 1,
 'どこ': 25,
 'どこ>一時集合場所': 1,
 'どこ>乳児': 1,
 'どこ>予防接種': 1,
 'どこ>公園': 1,
 'どこ>公園>分園': 1,
 'どこ>公園>分園>禁煙': 1,
 'どこ>場所': 1,
 'どこ>場所>母子手帳': 1,
 'どこ>場所>母子手帳>受け取り': 1,
 'どこ>妊婦健診': 1,
 'どこ>妊婦健診>乳児': 1,
 'どこ>妊婦健診>乳児>医療機関': 1,
 'どこ>妊婦健診>乳児>医療機関>契約': 1,
 'どこ>市内': 1,
 'どこ>市内>認定こども園': 1,
 'どこ>戸籍': 1,
 'どこ>支所': 1,
 'どこ>支所>出張所': 1,
 'どこ>支援': 1,
 'どこ>支援>子育て': 1,
 'どこ>支援>子育て>家庭': 1,
 'どこ>支援>子育て>家庭>センター': 1,
 'どこ>施設': 3,
 'どこ>施設>ところ': 1,
 'どこ>施設>ところ>赤ちゃん': 1,
 'どこ>施設>ところ>赤ちゃん>体重': 1,
 'どこ>

In [99]:
doc_all = open('document_v2.txt', 'r').readlines()
doc_all.append('')
len(doc_all)

3948

In [132]:
tag2line = {}
tag2line[''] = [1, 12] # Documentの2~12行目に対応

for tag in tqdm(list(sample_cnt.keys())):
    if tag == '':
        continue
    idx = 0
    level = 0
    line_pos = [-1, -1]
    tags = tag.split('>')
    for word in tags:
        while True:
            indent = doc_all[idx].rstrip().split('\t')
            if len(indent)==level+1 and indent[-1] == '<'+word+'>':
                level += 1
                break
            idx += 1
    line_pos[0] = idx
    idx += 1
    while idx < len(doc_all): 
#         if doc_all[idx].strip() == '':
#             line_pos[1]=idx
#             break
        if doc_all[idx].strip() != '' and doc_all[idx].rstrip().split('\t')[-1][0]=='<' and len(doc_all[idx].rstrip().split('\t'))<level+1:
            line_pos[1]=idx
            break
        idx += 1
    tag2line[tag] = line_pos

/Users/yuki/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/1423 [00:00<?, ?it/s]

In [65]:
line = "		<出生届>\n"
len(line.rstrip().split('\t'))

3

In [133]:
tag2line

{'': [1, 12],
 '10か月': [13, 20],
 '1歳': [20, 33],
 '1歳>6か月': [21, 25],
 '1歳>保健所': [25, 33],
 '3歳': [33, 46],
 '5才': [46, 53],
 '5才>息子': [47, 53],
 '5才>息子>最近': [48, 53],
 '7か月': [53, 60],
 'ADHD': [60, 66],
 'BCG': [66, 71],
 'LGBT': [71, 82],
 'QRコード': [82, 86],
 'いつ': [86, 99],
 'いつ>届出': [89, 94],
 'いつ>届出>出生届': [90, 94],
 'いつ>市役所': [94, 99],
 'いつ>市役所>年内': [96, 99],
 'おしめ': [99, 104],
 'がん検診': [104, 111],
 'こと': [111, 124],
 'こと>うち': [114, 118],
 'こと>産前': [118, 124],
 'こと>産前>産後': [119, 124],
 'こども食堂': [124, 129],
 'ごみ': [129, 145],
 'ごみ>分別': [140, 145],
 'ご当地グルメ': [145, 148],
 'とき': [148, 167],
 'とき>児童扶養手当証書': [149, 152],
 'とき>入院': [152, 158],
 'とき>入院>食事': [153, 158],
 'とき>入院>食事>療養': [154, 158],
 'とき>問い合わせ': [158, 167],
 'とき>問い合わせ>病気': [159, 167],
 'ところ': [167, 176],
 'ところ>リトミック': [168, 172],
 'ところ>放課後': [172, 176],
 'ところ>放課後>学童': [173, 176],
 'どこ': [176, 306],
 'どこ>一時集合場所': [177, 184],
 'どこ>乳児': [184, 189],
 'どこ>予防接種': [189, 197],
 'どこ>公園': [197, 203],
 'どこ>公園>分園': [198, 203],
 'どこ>公園

In [134]:
with open('tag2line.json', 'w') as f:
    json.dump(tag2line, f, indent=4, ensure_ascii=False)

In [150]:
all_data = {}

for i in range(len(questions)):
    all_data[i] = {}
    tag = idx2tags[i]
    ori_tag = copy.deepcopy(tag)
    while True:
        if sample_cnt[tag] > 1 or len(tag.split('>')) == 1:
            break
        else:
            tag = '>'.join(tag.split('>')[:-1])
    start, end = tag2line[tag]
    document = ''.join(doc_all[start:end]).replace('\t', '')
    if len(document) > 500:
        start, end = tag2line[ori_tag]
        document = ''.join(doc_all[start:end]).replace('\t', '')
    all_data[i]['document'] = document
    all_data[i]['question'] = questions[i]
    all_data[i]['answer'] = answers[i]

In [136]:
print("".join(doc_all[33:45]))

<3歳>
3歳児健診で尿が取れない場合でも、健診後、4歳になる前日までに健診受付時間内に、尿をお持ちいただければ追加で検査できます。
お子さんの成長に合わせてお持ちください。
また、健診の朝の尿が難しい場合は、健診会場でお取りいただくこともできます。
3歳児健診は子どもの3歳の誕生月に通知されますので、指定の保健相談所にて受診してください。
▼詳しくはこちら
（自治体HP内関連ページのURL）
健診の日時や場所はお住まいの地域により異なります。
次の健診の日時等は電話でお問い合わせください。
◆お問い合わせ
（自治体の担当課等の名称）
（電話番号）／（開庁時間）



In [111]:
tag2line['アカウント>配信']

[332, 335]

In [151]:
with open('all_train_data.json', 'w') as f:
    json.dump(all_data, f, indent=4, ensure_ascii=False)

In [152]:
all_data[1]

{'document': '<受け取り>\n母子手帳は、○○市役所本庁舎△△階××課窓口、◎◎出張所、………（その他の受け取り場所を適宜記載）………で受け取れます。\n▼詳しくはこちら\n（自治体HP内関連ページのURL）\n',
 'question': '母子手帳の受け取り場所はどこですか？',
 'answer': '母子手帳は、○○市役所本庁舎△△階××課窓口、◎◎出張所、………（その他の受け取り場所を適宜記載）………で受け取れます。\n▼詳しくはこちら\n（自治体HP内関連ページのURL）'}

In [139]:
idx2tags[1]

'どこ>場所>母子手帳>受け取り'

In [122]:
sample_cnt['どこ']

25

In [121]:
'>'.join('どこ>場所'.split('>')[:-1])

'どこ'

In [123]:
tag2line['どこ']

[176, 196]

In [153]:
all_data[103]

{'document': '<保育>\n保育室とは、●●県の定める設置基準を満たした、産休明けから2歳までのお子さんを預かる小規模な保育施設です。\n▼詳しくはこちら\n（自治体HP内関連ページのURL）\n<いくら>\n<延長>\n市立保育園の延長保育の保育料については、こちらをご確認ください。\n（自治体HP内関連ページのURL）\n私立保育園の延長保育の保育料については、保育園に直接お問い合わせください。\n<しおり>\n（該当するものがあれば記載してください。）\n例「保育のしおりとは各保育園で作成し、入園時等に配布している園のルール等を記載したものです。通っている保育園で入手できますので、各保育園にお問い合わせください。」\n<ところ>\n<延長>\n○○市で延長保育を実施している保育園についてはこちらをご覧ください。\n（自治体HP内関連ページのURL）\n<どこ>\n<認定こども園>\n（認定こども園の保育料の支払い先について記載してください。）\n例「認定こども園の保育料のお支払いは、直接認定こども園にお願いします。」\n<よう>\n<時間>\n（保育時間の説明を記載してください。）\n例「保育時間は、開園時間の中で、保護者の就労時間、通勤時間、家庭状況等を考慮し、必要最小限の保育時間で決定します。」\n<一時>\n<キャンセル>\n<当日>\n（キャンセルの場合の対応を記載してください。）\n例「一時保育をキャンセルされる場合は、各施設へお早めにご連絡ください。当日キャンセルの場合、キャンセル料が発生する場合があります。詳しくは各施設へお問い合わせください。\n▼詳しくはこちら\n（自治体HP内関連ページのURL）\n<制度>\n一時保育制度とは、子育てのリフレッシュを主な目的に、市内在住の小学校就学前までの子どもを預かる制度です。市内○か所の保育園で実施しています。\n料金の支払方法や申込方法等については、各施設へお問い合わせください。\n▼詳しくはこちら\n（自治体HP内関連ページのURL）\n<事業>\n<家庭>\n<保育ママ>\n家庭的保育事業とは、乳児又は幼児について、家庭的保育者の居宅その他の場所において、家庭的保育者による保育を行う事業です。児童福祉法第34条の15第2項により、市区町村の認可を受けて行うものと、認可を受けずに行うもの

In [144]:
len(all_data[103]['document'])

4725